In [1]:
!mkdir -p data/receipt_validation/train/receipt
!mkdir -p data/receipt_validation/train/not_receipt
!mkdir -p data/receipt_validation/val/receipt
!mkdir -p data/receipt_validation/val/not_receipt


In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:
img_size=224
batch_size=4

In [4]:
train_transform=transforms.Compose([
    transforms.Resize((img_size,img_size)),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform=transforms.Compose([
    transforms.Resize((img_size,img_size)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [5]:
train_dataset = datasets.ImageFolder(
    root="data/receipt_validation/train",
    transform=train_transform
)

val_dataset = datasets.ImageFolder(
    root="data/receipt_validation/val",
    transform=val_transform
)



train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False
)

In [6]:
images, labels = next(iter(train_loader))

print("Image batch shape:", images.shape)
print("Labels:", labels)
print("Class mapping:", train_dataset.class_to_idx)

Image batch shape: torch.Size([4, 3, 224, 224])
Labels: tensor([0, 1, 0, 0])
Class mapping: {'not_receipt': 0, 'receipt': 1}


In [7]:
import torch.nn as nn
from torchvision import models


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


model = models.resnet18(pretrained=True)


for param in model.parameters():
    param.requires_grad = False


Using device: cuda
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 44.7M/44.7M [00:00<00:00, 195MB/s]


In [8]:
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)


model = model.to(device)

In [9]:
dummy_input = torch.randn(1, 3, 224, 224).to(device)
dummy_output = model(dummy_input)

print("Dummy output shape:", dummy_output.shape)

Dummy output shape: torch.Size([1, 2])


In [10]:
import torch.optim as optim


criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


EPOCHS = 5

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    print("-" * 30)


    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)


        outputs = model(images)
        loss = criterion(outputs, labels)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    avg_train_loss = train_loss / len(train_loader)

    print(f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.4f}")


    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    avg_val_loss = val_loss / len(val_loader)

    print(f"Val Loss:   {avg_val_loss:.4f} | Val Acc:   {val_acc:.4f}")



Epoch 1/5
------------------------------
Train Loss: 0.8287 | Train Acc: 0.5000
Val Loss:   0.9305 | Val Acc:   0.2000

Epoch 2/5
------------------------------
Train Loss: 0.6663 | Train Acc: 0.5500
Val Loss:   0.8421 | Val Acc:   0.3000

Epoch 3/5
------------------------------
Train Loss: 0.5234 | Train Acc: 0.8500
Val Loss:   0.7125 | Val Acc:   0.4000

Epoch 4/5
------------------------------
Train Loss: 0.5389 | Train Acc: 0.7500
Val Loss:   0.5993 | Val Acc:   0.7000

Epoch 5/5
------------------------------
Train Loss: 0.4298 | Train Acc: 0.9000
Val Loss:   0.5450 | Val Acc:   0.9000
